In [60]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.linear_model import Ridge

data = pd.read_csv('C:/Users/JP/Documents/School/Practicum/Github/Practicum/data/dataset2.csv')

df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
df = df.set_index('Date')
df=df[df.index>'2017-05-30']

train=df[df.index<'2017-09-01']

features = ['TBondsOpenValue', 'UnemploymentValue', 'BTCOpen', 'BTCOpenTM1', 'BTCOpenTM2',\
        'BTCOpenTM3', 'BTCOpenTM4', 'BTCOpenTM5', 'BTCOpenTM6', 'BTCOpenTM7','GDP','SandPValue',\
            'ETHOpenTM1', 'ETHOpenTM2',\
        'ETHOpenTM3', 'ETHOpenTM4', 'ETHOpenTM5', 'ETHOpenTM6', 'ETHOpenTM7']

train=train.dropna()


dtrain = xgb.DMatrix(train.loc[:, features].values, \
                     label = train.loc[:, 'ETHOpen'].values)

params = {}
params['booster']  = 'gbtree'
params['objective'] = 'reg:linear'
params['max_depth'] = 6
params['subsample'] = 0.8
params['colsample_bytree'] = 0.8
params['silent'] = 1
params['eval_metric'] = 'rmse'
num_round = 50
eval_list  = [(dtrain,'train')]

train['Date'] = train.index.values

print('Training xgb model:')
bst = xgb.train(params, dtrain, num_round, eval_list)

print('Trainign Ridge Regression')
lr = Ridge()
lr.fit(train.loc[:, features].values, \
       train.loc[:, 'ETHOpen'].values)

test = df[df.index>='2017-09-01']
test=test.dropna()

while True:
    dtest = xgb.DMatrix(test[features].values)
    xgb_pred = bst.predict(dtest)
    lr_pred = lr.predict(test[features].values)
    test['ETHOpen'] = 0.2*xgb_pred+0.8*lr_pred

    target = train['ETHOpen']
    
    done = 1
    
    if done:
        print("Prediction: {}".format(test['ETHOpen']))
        break



training xgb model...
[0]	train-rmse:199.135
[1]	train-rmse:144.441
[2]	train-rmse:104.401
[3]	train-rmse:75.0942
[4]	train-rmse:54.8394
[5]	train-rmse:39.909
[6]	train-rmse:29.0933
[7]	train-rmse:21.4535
[8]	train-rmse:16.1608
[9]	train-rmse:12.3654
[10]	train-rmse:9.40966
[11]	train-rmse:7.69997
[12]	train-rmse:6.03475
[13]	train-rmse:4.85785
[14]	train-rmse:3.90114
[15]	train-rmse:3.36007
[16]	train-rmse:2.67474
[17]	train-rmse:2.20594
[18]	train-rmse:1.83359
[19]	train-rmse:1.61321
[20]	train-rmse:1.35221
[21]	train-rmse:1.20292
[22]	train-rmse:1.06671
[23]	train-rmse:0.909228
[24]	train-rmse:0.798715
[25]	train-rmse:0.715271
[26]	train-rmse:0.659335
[27]	train-rmse:0.573475
[28]	train-rmse:0.498174
[29]	train-rmse:0.441628
[30]	train-rmse:0.397919
[31]	train-rmse:0.355675
[32]	train-rmse:0.309196
[33]	train-rmse:0.281217
[34]	train-rmse:0.246979
[35]	train-rmse:0.216129
[36]	train-rmse:0.19393
[37]	train-rmse:0.167735
[38]	train-rmse:0.157404
[39]	train-rmse:0.136133
[40]	train-rm

In [64]:
df[df.index<'2017-09-13']['ETHOpen'].tail()

Date
2017-09-06    317.94
2017-09-07    338.92
2017-09-08    335.37
2017-09-11    299.21
2017-09-12    297.95
Name: ETHOpen, dtype: float64